# Step 1: Loading the data

We will follow a similar process in this step, only now the data set comes from different sources. 

In [1]:
import pandas as pd

We will load a data set that integrates the data we have from the different sources. cell. 

In [2]:
df = pd.read_excel('data/mixed_data.xls')

The file is available on Nestor.

We start our exploratory analysis by inspecting the size of the data file

In [3]:
df.shape

(30000, 26)

So this file has 30000 records and 26 fields (columns). Note that this is one more column than before. 

# Step 2: Check Data Integrity

As before, the data set contains monthly credit card account data, for a period of six months. Let's perform a quality check to ensure we have the data for the accounts as expected. The account ID distinguishes one account from the other. We can check unique IDs with Pandas with the function `.nunique()`. But first let's check our data structure. To do so we first build an Index of he different columns in the table. We will use the .columns method of the pandas DataFrame to see the column names. 

In [4]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'DOB',
       'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

Can you see the heading for the additional column?

This is the DOB column. 


We then check the column headings

In [5]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,DOB,PAY_1,PAY_2,PAY_3,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,b15e78fe-a223,500000,1,1,1,42,1900-01-01,-2,-1,0,...,6420,49240,6772,31305,43864,6453,49470,3723,103686,0
1,689ae71c-3f4b,30000,1,2,1,36,1900-01-01,1,-1,-1,...,1170,780,0,780,0,1170,0,0,0,0
2,3dc1d96e-35c2,50000,1,1,1,0,1981-10-21,0,0,0,...,31240,20343,12349,2017,3000,2000,3007,1003,1500,0
3,f786754a-eb0b,50000,1,3,1,0,1982-10-22,Not available,-1,-1,...,1961,1261,2681,13196,2866,1961,1261,2681,1261,0
4,e6a3c5a6-7647,200000,2,1,2,36,1900-01-01,-2,-2,-2,...,0,0,0,0,0,0,0,0,0,1


Now we see: we have a DOB column which is likely to be Date of Birth. 
Indeed we consult our sources and this is confirmed. 
But something is not quite right with the data now. We see that for some records we have the 'AGE'column, while for some other we have the "DOB". Additionally, do you see something additional which is not quite right?
As our data came from different sources, one of the sources was recording the age, but for this one the DOB contains an invalid date, while the opposite happens for the other source. 
We need to do something about it. But before doing so, let's repeat our previous steps. 

In [6]:
df.shape

(30000, 26)

And now we check the unique IDs

In [7]:
df['ID'].nunique()

29687

That's interesting! We have fewer unique IDs than rows, so we clearly hae duplicates! We can check the number of occurences of each ID by counting them, as follows: 

In [8]:
id_counts = df['ID'].value_counts()
id_counts.head()

4fcef74d-0b3d    2
a7e199e1-2c44    2
a9ce8636-9a88    2
d9dafe9c-b497    2
3367c92b-b1ca    2
Name: ID, dtype: int64

In [9]:
id_counts.value_counts()

1    29374
2      313
Name: ID, dtype: int64

The number of records is not different than before: We have 29374 IDs which appear only once but we also hae 313 which appear twice! 

# Step 3: Continue with Data Integrity

We will now examine in mode detail the duplicate IDs. To do so, let's create a Boolean mask to locate the duplicate IDs. We name this 'mask dupe_mask'. Let's display the first five elements.

In [10]:
dupe_mask = id_counts == 2

In [11]:
dupe_mask[0:5]

4fcef74d-0b3d    True
a7e199e1-2c44    True
a9ce8636-9a88    True
d9dafe9c-b497    True
3367c92b-b1ca    True
Name: ID, dtype: bool

You can change the '5'in the command above with another number and observe a different number of records. Let's keep it '5'now and inspect the duplicate ID records. 

In [12]:
id_counts.index[0:5]

Index(['4fcef74d-0b3d', 'a7e199e1-2c44', 'a9ce8636-9a88', 'd9dafe9c-b497',
       '3367c92b-b1ca'],
      dtype='object')

To be able to access and inspect these records, let's store their ID numbers in a variable named 'dupe_ids' and inspect the first 10 such records:  

In [13]:
dupe_ids = id_counts.index[dupe_mask]

In [14]:
dupe_ids[:10]

Index(['4fcef74d-0b3d', 'a7e199e1-2c44', 'a9ce8636-9a88', 'd9dafe9c-b497',
       '3367c92b-b1ca', '26d4a6eb-8abe', '98395d08-9643', '823267c1-561b',
       '82da3aa5-97a6', '26bde6da-f148'],
      dtype='object')

Let's double check the cardinality of the dupe_ids variable (i.e. how many records it contains). We use 'len' for this: 

In [15]:
dupe_ids = list(dupe_ids)
len(dupe_ids)

313

That's right - as expected. So we focus again on the first 5. 

In [16]:
dupe_ids[0:5]

['4fcef74d-0b3d',
 'a7e199e1-2c44',
 'a9ce8636-9a88',
 'd9dafe9c-b497',
 '3367c92b-b1ca']

Now that we have selected the records with the duplicate IDs, we wish to inspect their data in more detail. Is there anything different in the values of these records which have identical IDs? Our approach to do so is this: we first focus on the data records with the first 3 duplicate IDs, as a sample. First we focus on finding the rows which contain these 3 IDs. The first three IDs are dupe_ids[0:3]. 

We will use the '.isin' method to create another logical mask. We will pass the earlier selected list of IDs through this mask and apply this on the larger dataframe to display the rows that have this ID. We use the .loc method for locating these IDs. The '.isin' method is NESTED within the .loc statement: this indexes the datafame to select the location of all rows containing a True value in the logical mask. The : in the second argument of the loc implies that all columns will be seleted. So in this way we filter the Dataframe so as to view all the columns for the first three duplicate IDs. This is not very comlex filtering but still it is a sign of the compactness of the Python language that a single command with this nested structure produces this outcome. 

In [17]:
df.loc[df['ID'].isin(dupe_ids[0:3]),:]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,DOB,PAY_1,PAY_2,PAY_3,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1742,4fcef74d-0b3d,60000,1,2,2,0,1995-11-04,2,2,2,...,120142,60428,58192,3000,2746,0,2297,1875,1158,1
6396,a9ce8636-9a88,20000,2,2,1,34,1900-01-01,Not available,0,0,...,10926,17536,2090,2000,3000,3000,7000,2090,0,0
17714,4fcef74d-0b3d,0,0,0,0,0,NaT,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27248,a7e199e1-2c44,0,0,0,0,0,1900-01-01,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28847,a7e199e1-2c44,100000,1,2,2,0,1990-10-30,0,0,0,...,25362,26318,42484,6363,4386,5362,4344,20000,4382,0
29669,a9ce8636-9a88,0,0,0,0,0,1900-01-01,0,0,0,...,0,0,0,0,0,0,0,0,0,0


This gets interesting. For each pair of rows with duplicate IDs, on row contains potentially valid data, but the other one only contains 0s! It is most likley that a data clearning process would be expected to delete all such eroneous data (you can't for example have an age of 0 or a credit limit of 0 and stil be part of the data processing! If we attempted to find the rows with all data values being zeros we would have failed, but if we seek to find the data records that apart from the ID value, all other variable values are 0s, this might work much better. We will use the == operator to make an asignment to a variable that will be used as a mask. 


In [18]:
df.shape

(30000, 26)

In [19]:
df_zero_mask = df == 0

We wil use the df_zero_mask to check for 0 values in all other columns apart from the ID one (which we know that does have non-0 values). So we will use all axes in the 'dimensions' of the data set, apart from the first one.

In [20]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)

In [21]:
sum(feature_zero_mask)

0

That's no good! Now that we have data from different sources, the simple rule we had last time does not seem to work. 

In [22]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy()

In [23]:
df_clean_1.shape

(30000, 26)

In [24]:
df_clean_1['ID'].nunique()

29687

So we are exactly where we started. We need to change approach. We first have to deal with resolving the data from the different sources .